# Notebook to generate one video per bee, cropped to the area around the bee, for videos in the Konstanz dataset (scented bee petri dish videos)

- Pose data is exported to CSV-files along with the videos (I only generated videos and exported pose data to CSV for some of the videos from the hex-group)
- This notebook is currently working with pixel values configured for the 1920x1080 videos but it could be adapted for the smaller videos in the same way as in *Trophallaxis_video_generation_Konstanz_data.ipynb*, by multiplying all pixels values with a ratio adjustment value

In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import subprocess
from concurrent.futures import ThreadPoolExecutor as PoolExecutor
import skimage.io
import imageio
from datetime import datetime
import bb_behavior.utils.images
from typing import Tuple
import cv2
import joblib
import matplotlib.pyplot as plt
import time
import math
import glob
import logging
import csv
from sortedcontainers import SortedDict
from video_utils import CustomVideoManager

In [ ]:
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

logging.basicConfig(
    filename='indiv_video_gen_konst_data.log',
    level=logging.DEBUG,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

# write info to console
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.INFO)
console_handler.setFormatter(logging.Formatter(
    '%(levelname)s - %(message)s'
))
logging.getLogger().addHandler(console_handler)
logger = logging.getLogger(__name__)
logger.info("Starting video generation ...")

## Set global variables

| Variable | Explanation |
| --- | --- |
| GENERATE_VIDEOS | Whether to generate videos; if False then only the CSV for the pose data in the videos is generated (much faster) |


In [ ]:
# set this to False if you want to only process and output the pose data
GENERATE_VIDEOS = True


VIDEO_ROOT = '/mnt/windows-ssd/BeesData/scented_bees_video_files/'
CACHE_PATH = '/mnt/windows-ssd/BeesData/tmp/'
POSTURE_FILES_PATH = '/mnt/windows-ssd/BeesData/scented_bees_posture_files/'
VIDEO_OUTPUT_PATH = '/mnt/windows-ssd/BeesData/videos_single_bees/'
MARKER_DATA_OUTPUT_PATH = '/mnt/windows-ssd/trophallaxis_detection_code/data/marker_data_konstanz'

N_JOBS = 16     # number of parallel jobs for frame extraction
FRAME_RATE = 50
FRAMES_PER_MINUTE = 60 * 50
# cut off the first five minutes of frames because bees are still waking up
MINUTES_CUTOFF_START = 5
FRAME_CUTOFF_START = MINUTES_CUTOFF_START * FRAMES_PER_MINUTE

BODYPARTS_HEADER = [
    'bodyparts', 'left_antenna', 'left_antenna', 'left_antenna',
    'right_antenna', 'right_antenna', 'right_antenna',
    'head', 'head', 'head',
    'thorax', 'thorax', 'thorax',
    'abdomen_tip', 'abdomen_tip', 'abdomen_tip'
]
COORDS_HEADER = [
    'coords', 
    'x', 'y', 'likelihood', 
    'x', 'y', 'likelihood', 
    'x', 'y', 'likelihood',
    'x', 'y', 'likelihood', 
    'x', 'y', 'likelihood'
]
COLS_BODYPARTS_EXPORT = [
    "left_antenna_x", "left_antenna_y", "left_antenna_conf",
    "right_antenna_x", "right_antenna_y", "right_antenna_conf",
    "head_x", "head_y", "head_conf",
    "thorax_x", "thorax_y", "thorax_conf",
    "abdomen_tip_x", "abdomen_tip_y", "abdomen_tip_conf"
]

#############################################################################################
# wcentroid is based on centroid weighted by pixel values, 
# pcentroid is based on posture centroid (the center of the midline)
# centroid is center of mass of all thresholded pixels
# nothing after the hashtag means based on head position
#############################################################################################
COLS_TO_EXCLUDE = [
    'tracklets','tracklet_vxys','video_size', 'id', 'frame_rate', 
    'ACCELERATION#pcentroid', 'AX', 'ANGULAR_A#centroid',#'ACCELERATION#wcentroid',
    'ANGULAR_V#centroid', 'BORDER_DISTANCE#pcentroid', 'AY', 'MIDLINE_OFFSET',
    'SPEED#wcentroid', 'SPEED#pcentroid', 'SPEED', 'VX', 'VY', 'X#wcentroid', 
    'Y#wcentroid', 'midline_length', 'midline_segment_length', 'midline_x', 
    'midline_y', 'missing', 'normalized_midline', 'num_pixels', 'timestamp'
]

## Build the dataframe from tracking and pose data

Pose data fields: <br>

poseX0, poseY0: left antenna <br>
poseX1, poseY1: right antenna <br>
poseX2, poseY2: proboscis <br>
poseX3, poseY3: head <br>
poseX4, poseY4: thorax <br>
poseX5, poseY5: abdomen (the tip) <br>

In [ ]:
def build_dataframe() -> pd.DataFrame:
    frame_id_offset = 0
    video_names = sorted(glob.glob(POSTURE_FILES_PATH + 'hex_*.npz'))
    for file_idx, file in enumerate(video_names):
        data = np.load(file)
        keys = data.files

        # Figure out how many rows (we’ll use 'time' as canonical—but any 1D key works)
        n = data['time'].shape[0]

        columns = {}

        for k in keys:
            if k in COLS_TO_EXCLUDE:
                continue
            v = data[k]
            
            # Scalar → broadcast to length n
            if np.ndim(v) == 0 or k == 'cm_per_pixel':
                columns[k] = np.repeat(v.item(), n)
            # 1-D array (including object-dtype) → straight in
            elif v.ndim == 1:
                columns[k] = v
            # Multi-D numeric array → flatten trailing dims into separate columns
            else:
                # reshape to (n_rows, -1)
                flat = v.reshape(n, -1)
                for i in range(flat.shape[1]):
                    columns[f"{k}_{i}"] = flat[:, i]

        # Build the DataFrame
        df_file = pd.DataFrame(columns)
        data_filename = (file.split("/")[-1]).split(".")[0]
        df_file['data_filename'] = data_filename

        # X and Y are in cm (convert to px)
        df_file = df_file.rename(columns={'cm_per_pixel': 'cm_per_px'})
        cm_per_px = df_file['cm_per_px'].iloc[0]
        df_file['x_pixels'] = np.round(df_file['X'] / cm_per_px)
        df_file['y_pixels'] = np.round(df_file['Y'] / cm_per_px)

        df_file['bee_id'] = data['id'][0]
        df_file['frame_index'] = df_file['frame'].astype(int)
        df_file = df_file.rename(columns={
            'ANGLE': 'orientation', 
            'poseX0': 'left_antenna_x', 'poseY0': 'left_antenna_y',
            'poseX1': 'right_antenna_x', 'poseY1': 'right_antenna_y',
            'poseX2': 'proboscis_x', 'poseY2': 'proboscis_y',
            'poseX3': 'head_x', 'poseY3': 'head_y',
            'poseX4': 'thorax_x', 'poseY4': 'thorax_y',
            'poseX5': 'abdomen_tip_x', 'poseY5': 'abdomen_tip_y',
            'ACCELERATION#wcentroid': 'accel'
        })
        df_file = df_file.drop(columns='frame')

        df_file = df_file[df_file.frame_index >= FRAME_CUTOFF_START]
        
        frames_cnt = len(df_file['frame_index'])
        df_file['frame_id'] = df_file['frame_index'] + frame_id_offset
        if (file_idx+1) % 4 == 0:
            frame_id_offset += frames_cnt

        df_files.append(df_file)
    df = pd.concat(df_files)
    df['video_filename'] = VIDEO_ROOT + df['data_filename'].str.split('_fish').str[0] + ".mp4"
    df[['left_antenna_conf', 'right_antenna_conf', 'proboscis_conf', 'head_conf', 'thorax_conf', 'abdomen_tip_conf']] = 1.0
    return df

df_files = []
df = build_dataframe()

print(df.keys())
print(len(df))

## Combine the tracking dataframe with the video file dataframe

The Custom video manager is used for caching and extracting frames from videos as well as saving frames to videos. It is partly adapted from a notebook by Jacob Davidson

In [ ]:
video_manager = CustomVideoManager(VIDEO_ROOT, CACHE_PATH, VIDEO_OUTPUT_PATH, max_workers=16)
video_manager.clear_video_cache()

videos_df = video_manager.get_all_video_files()
df = pd.merge(df, videos_df, on='video_filename')

## Get the cropped images for the bees with the corresponding body masks (for other bees)
This cell was adapted from https://github.com/nebw/unsupervised_behaviors/blob/master/unsupervised_behaviors/data.py


In [ ]:
def process_frames(
    detections: pd.DataFrame,
    video_manager: CustomVideoManager,
    image_size_px: int = 272,
    image_crop_px: int = 40,
    body_center_offset_px: int = 35,
    body_mask_length_px: int = 96,
    body_mask_width_px: int = 64,
    egocentric: bool = True,
    generate_videos: bool = True,
    use_clahe: bool = True,
    clahe_kernel_size_px: int = 25,
    n_jobs: int = -1
) -> Tuple[np.ndarray, np.ndarray, pd.DataFrame]:
    """Get cached images and for each bee, crop them to the area around this bee and optionally rotate 
       image region so that the bee is egocentrically aligned. Create body mask to hide other bees (could 
       be removed). Adjust keypoint data to this new image region by using translation and rotation.

    Args:
        detections (pd.DataFrame): Dataframe with detections.
        video_manager (CustomVideoManager): Manages cache.
        image_size_px (int, optional): Image size before cropping. Defaults to 272.
        image_crop_px (int, optional): Crop amount after rotation. Defaults to 40.
        body_center_offset_px (int, optional): Offset between body center and bee coordinates (head). Defaults to 35.
        body_mask_length_px (int, optional): Length of body mask. Defaults to 96.
        body_mask_width_px (int, optional): Width of body mask. Defaults to 64.
        egocentric (bool, optional): Whether to rotate the frames so that the focus bee is egocentrically aligned. 
            Defaults to True.
        generate_videos (bool, optional): Whether to generate video clips. Defaults to True.
        use_clahe (bool, optional): Process entire frame using CLAHE. Defaults to True.
        clahe_kernel_size_px (int, optional): Kernel size for CLAHE. Defaults to 25.
        n_jobs (int, optional): Number of parallel jobs for processing. Defaults to -1.

    Returns:
        Tuple[np.ndarray, np.ndarray, pd.DataFrame]: Images, body masks and the dataframe adjusted to the new crop region
    """

    def rotate_crop_img(image: np.ndarray, rotation_deg: float) -> np.ndarray:
        image = skimage.transform.rotate(image, rotation_deg)
        image = image[image_crop_px:-image_crop_px, image_crop_px:-image_crop_px]
        return image
    

    def rotate_pts_around_center(
        pts: list[list[float]], 
        rot_center: Tuple[float, float], 
        degrees: float
    ) -> np.ndarray:
        """Rotate keypoints around a point (crop center)

        Args:
            pts (list[list[float]]): list of keypoints Nx2 like [[x1,y1], [x2,y2], ...]
            rot_center (Tuple[float, float]): Point to rotate around
            degrees (float): Amount to rotate in clockwise direction

        Returns:
            np.ndarray: Rotated points in the same form as pts
        """

        pts = np.asarray(pts, dtype=float)
        cx, cy = rot_center
        theta = np.deg2rad(-degrees)
        c = np.array([cx, cy])
        R = np.array([[np.cos(theta), -np.sin(theta)],
                    [np.sin(theta),  np.cos(theta)]])
        
        # exclude inf values in marker data from rotation (because they can become nan)
        mask = np.isfinite(pts).all(axis=1)
        result = pts.copy()
        finite_pts = pts[mask]
        result[mask] = (R @ (finite_pts - c).T).T + c

        return result

    
    def process_pose_data_and_extract_images_from_frame(
        frame_detections: pd.DataFrame, 
        frame_path: str,
        fetch_images: bool = True,
        egocentric: bool = True
    ) -> Tuple[np.ndarray, np.ndarray, list[pd.Series]]:
        images = []
        body_masks = []
        rows = []

        assert frame_detections.frame_id.nunique() == 1

        if fetch_images:
            frame = imageio.v3.imread(frame_path, plugin="opencv", colorspace="GRAY")
            if use_clahe:
                frame = skimage.exposure.equalize_adapthist(frame, kernel_size=(clahe_kernel_size_px, clahe_kernel_size_px))

        for _, row in frame_detections.iterrows():
            body_center_adj_x = np.cos(row.orientation) * body_center_offset_px
            body_center_adj_y = np.sin(row.orientation) * body_center_offset_px

            if egocentric:
                # so that bee is facing to the right (row.orientation + np.pi / 2 for facing upwards)
                rotation_deg = (1 / (2 * np.pi)) * 360 * row.orientation
            else:
                rotation_deg = 0


            # ---- transform marker locations to new cropped and rotated frame ----
            center_of_crop = (image_size_px - 2 * image_crop_px) / 2

            # calc. translated keypoints (adjust for body center offset)
            la_x_trans = row.left_antenna_x - (row.x_pixels - center_of_crop) + body_center_adj_x
            la_y_trans = row.left_antenna_y - (row.y_pixels - center_of_crop) + body_center_adj_y
            ra_x_trans = row.right_antenna_x - (row.x_pixels - center_of_crop) + body_center_adj_x
            ra_y_trans = row.right_antenna_y - (row.y_pixels - center_of_crop) + body_center_adj_y
            prob_x_trans = row.proboscis_x - (row.x_pixels - center_of_crop) + body_center_adj_x
            prob_y_trans = row.proboscis_y - (row.y_pixels - center_of_crop) + body_center_adj_y
            head_x_trans = row.head_x - (row.x_pixels - center_of_crop) + body_center_adj_x
            head_y_trans = row.head_y - (row.y_pixels - center_of_crop) + body_center_adj_y
            thor_x_trans = row.thorax_x - (row.x_pixels - center_of_crop) + body_center_adj_x
            thor_y_trans = row.thorax_y - (row.y_pixels - center_of_crop) + body_center_adj_y
            abdo_x_trans = row.abdomen_tip_x - (row.x_pixels - center_of_crop) + body_center_adj_x
            abdo_y_trans = row.abdomen_tip_y - (row.y_pixels - center_of_crop) + body_center_adj_y
            
            # rotate the points
            rotated_pts = rotate_pts_around_center(pts=[
                [la_x_trans, la_y_trans],
                [ra_x_trans, ra_y_trans],
                [prob_x_trans, prob_y_trans],
                [head_x_trans, head_y_trans],
                [thor_x_trans, thor_y_trans],
                [abdo_x_trans, abdo_y_trans]
            ], rot_center=(center_of_crop, center_of_crop), degrees=rotation_deg)

            row.left_antenna_x = rotated_pts[0][0]
            row.left_antenna_y = rotated_pts[0][1]
            row.right_antenna_x = rotated_pts[1][0]
            row.right_antenna_y = rotated_pts[1][1]
            row.proboscis_x = rotated_pts[2][0]
            row.proboscis_y = rotated_pts[2][1]
            row.head_x = rotated_pts[3][0]
            row.head_y = rotated_pts[3][1]
            row.thorax_x = rotated_pts[4][0]
            row.thorax_y = rotated_pts[4][1]
            row.abdomen_tip_x = rotated_pts[5][0]
            row.abdomen_tip_y = rotated_pts[5][1]

            rows.append(row.values)
                

            if fetch_images:
                center_x = row.x_pixels - body_center_adj_x
                center_y = row.y_pixels - body_center_adj_y

                # assert center point always within frame (even after trajectory extrapolation)
                center_x = max(0, center_x)
                center_x = min(frame.shape[1] - 1, center_x)
                center_y = max(0, center_y)
                center_y = min(frame.shape[0] - 1, center_y)

                center = np.array((center_x, center_y))

                image = bb_behavior.utils.images.get_crop_from_image(
                    center, frame, width=image_size_px, clahe=False
                )
                image = (rotate_crop_img(image, rotation_deg) * 255).astype(np.uint8)

                body_mask = np.zeros_like(frame)
                body_coords = skimage.draw.ellipse(
                    center[1],
                    center[0],
                    body_mask_length_px,
                    body_mask_width_px,
                    rotation=-(row.orientation - np.pi / 2),
                    shape=frame.shape,
                )
                body_mask[body_coords] = 1
                body_mask = (
                    bb_behavior.utils.images.get_crop_from_image(
                        center, body_mask, width=image_size_px, clahe=False
                    )
                    == 255
                )
                body_mask = rotate_crop_img(body_mask, rotation_deg) > 0.5

                images.append(image)
                body_masks.append(body_mask)
        return images, body_masks, rows

    if len(detections.index) == 0:
        return

    images = []
    body_masks = []
    rows = []

    logger.debug(f'Detection count in video: {len(detections.index)}')
    
    detections_by_frame = detections.groupby("frame_id")

    # preload file paths because video_manager can't be used in parallel.
    frame_paths = []
    for _, frame_detections in detections_by_frame:
        frame_paths.append(video_manager.get_frame_id_path(frame_detections.frame_id.iat[0]))
    
    logger.debug(f"Processing {len(frame_paths)} cached images ...")
    parallel = joblib.Parallel(prefer="processes", n_jobs=n_jobs)(
        joblib.delayed(process_pose_data_and_extract_images_from_frame)(
            frame_detections, 
            frame_path, 
            fetch_images=generate_videos, 
            egocentric=egocentric
        )
        for (_, frame_detections), frame_path in zip(detections_by_frame, frame_paths)
    )

    logger.debug("Processing of cached images complete.")

    for results in parallel:
        images += results[0]
        body_masks += results[1]
        rows += results[2]

    images = np.stack(images) if len(images) > 0 else np.array([])
    body_masks = np.stack(body_masks) if len(body_masks) > 0 else np.array([])
    detections = pd.DataFrame(np.stack(rows), columns=detections.columns)

    return images, body_masks, detections

## Utility functions

In [ ]:
def interpolate_short_inf_runs(s: pd.Series, max_gap: int = 10) -> pd.Series:
    """Interpolate missing data in regions, where the number of consecutive 
       missing points is <= max_gap

    Args:
        s (pd.Series): Input series
        max_gap (int, optional): Maximum number of consecutive points, which get interpolated. 
            Defaults to 10.

    Returns:
        pd.Series: Interpolated series
    """
    s = s.replace(np.inf, np.nan).copy()
    isnan = s.isna()

    # cumsum() increments at every number (at every non-NaN), so all consecutive NaNs get the same run id
    run_ids = (~isnan).cumsum()

    # dict{run_id: run_length}
    run_length_dict = isnan.groupby(run_ids).sum().to_dict()

    # map each element to its run length
    run_lengths = run_ids.map(run_length_dict)

    short_run_mask = isnan & (run_lengths <= max_gap)

    interpolated = s.interpolate(method='linear')
    interpolated[~short_run_mask & isnan] = np.nan
    return interpolated


def interpolate_missing_vals(df_video: pd.DataFrame) -> pd.DataFrame:
    """Interpolate missing data (some of it conditionally)

    Args:
        df_video (pd.DataFrame): Input dataframe

    Returns:
        pd.DataFrame: Interpolated dataframe
    """

    # interpolate inf values in orientation, coordinates and acceleration
    cols_interpol = ['orientation', 'x_pixels', 'y_pixels', 'accel']
    df_video[cols_interpol] = (
        df_video
        .groupby('bee_id', sort=False)[cols_interpol]
        # use transform to use groupby and apply the methods inplace
        .transform(lambda x: x.replace(np.inf, np.nan).interpolate(method='linear'))
    )
    # replace leading or trailing nans with 0
    df_video[['accel']] = df_video[['accel']].fillna(0)

    cols_cond_interpol = [
        'left_antenna_x', 'left_antenna_y', 
        'right_antenna_x', 'right_antenna_y',
        'proboscis_x', 'proboscis_y', 
        'head_x', 'head_y', 
        'thorax_x', 'thorax_y', 
        'abdomen_tip_x', 'abdomen_tip_y'
    ]

    for col in cols_cond_interpol:
        df_video[col] = (
            df_video
            .groupby('bee_id', sort=False)[col]
            # interpolate where run of 'inf'-values is 10 or shorter, otherwise inf becomes nan
            .transform(lambda x: interpolate_short_inf_runs(x, max_gap=10))
        )
    
    return df_video

## Cache frames, extract and process them and write to videos
Main tasks:
1. Cache all frames from the videos by iterating over each minute. 
2. From the cached frames extract all cropped and rotated frames showing the individual bees in the frames.
3. Apply the mask and write the extracted frames to one-minute-long videos for each individual bee.
4. Write the pose data adjusted to the new cropped and rotated frames to CSV-files

In [ ]:
def generate_vids_export_csv_one_minute(
    video_name_short: str, 
    minute: int, 
    csv_writers: list, 
    images_masked: np.ndarray, 
    generate_videos: bool, 
    df_minute: pd.DataFrame
):
    """For each bee writes one minute of original video (adjusted to the bee) to a video and
       writes pose data for the bee to CSV-file

    Args:
        video_name_short (str): Short string from the original video (video-group + index)
        minute (int): Minute of the original video
        csv_writers (list): List of 4 CSV-writers, one for each bee
        images_masked (np.ndarray): Extracted images multiplied with masks
        generate_videos (bool): Whether to generate video clips
        df_minute (pd.DataFrame): Dateframe for this minute
    """

    # sort the df by bee_id and timestamp and apply the same sorting to the images
    df_minute_sorted = df_minute.sort_values(['bee_id', 'time'])
    sorting_indices = df_minute_sorted.index

    if generate_videos:
        images_sorted = [images_masked[i] for i in sorting_indices]

    df_minute_grouped = df_minute_sorted.groupby('bee_id', sort=False)
    # dict{bee_id -> indices}
    df_minute_grouped_indices = df_minute_grouped.indices
    df_minute_grouped_indices_iter = iter(df_minute_grouped_indices)
    
    for bee_id, df_bee_minute in df_minute_grouped:
        if generate_videos:
            indices_by_bee_id = df_minute_grouped_indices.get(next(df_minute_grouped_indices_iter))

            start_time = df_bee_minute.time.iat[0]
            end_time = df_bee_minute.time.iat[-1]
            filename = "_".join((video_name_short, "bee" + str(bee_id), (str(math.floor(start_time)) + "--" + str(math.ceil(end_time))) + ".mp4"))
            images_bee = [images_sorted[i] for i in indices_by_bee_id]
            video_manager.write_to_video(images=images_bee, filename=filename, frame_rate=FRAME_RATE)

        # write to csv
        coords_data = df_bee_minute[COLS_BODYPARTS_EXPORT].to_numpy()
        data_rows = [[minute * FRAMES_PER_MINUTE + i, *row_arr] for i, row_arr in enumerate(coords_data)]
        csv_writers[bee_id].writerows(data_rows)



def process_minute(
    video_file: str, 
    video_name_short: str, 
    df_video: pd.DataFrame, 
    rows_per_bee: int, 
    remainder_last_minute: int, 
    csv_writers: list, 
    minute: int, 
    generate_videos: bool,
    total_minutes: int
):
    """Process one minute of the original video

    Args:
        video_file (str): Original video file (including path)
        video_name_short (str): Short string from the original video (video-group + index)
        df_video (pd.DataFrame): Dateframe for original video
        rows_per_bee (int): Total rows per bee in the dataframe for the original video (number of frames per bee)
        remainder_last_minute (int): Remainder frames/rows in the last minute of the original video
        csv_writers (list): List of 4 CSV-writers, one for each bee
        minute (int): Minute of the original video (0-indexed)
        generate_videos (bool): Whether to generate video clips
        total_minutes (int): Number of total minutes rounded up in the original video to process 
            (excludes first five minutes, where bees are 'waking up')
    """

    logger.info(f"Processing minute {minute+1}")
    start_idx = minute * FRAMES_PER_MINUTE
    end_idx = start_idx + 60 * FRAME_RATE
    if minute == total_minutes - 1:
        end_idx = start_idx + remainder_last_minute

    df_minute_bee0 = df_video.iloc[start_idx:end_idx]
    df_minute_bee1 = df_video.iloc[rows_per_bee + start_idx:rows_per_bee + end_idx]
    df_minute_bee2 = df_video.iloc[2 * rows_per_bee + start_idx:2 * rows_per_bee + end_idx]
    df_minute_bee3 = df_video.iloc[3 * rows_per_bee + start_idx:3 * rows_per_bee + end_idx]
    df_minute = pd.concat([df_minute_bee0, df_minute_bee1, df_minute_bee2, df_minute_bee3])
    
    if generate_videos:
        video_manager.cache_frames(
            frame_ids=np.sort(df_minute['frame_id'].unique()), 
            video_name=video_file, 
            frame_indices=np.sort(df_minute['frame_index'].unique())
        )

    # get images and masks for all detections in one video
    images, body_masks, df_minute_after = process_frames(
        detections=df_minute, 
        video_manager=video_manager, 
        generate_videos=generate_videos, 
        n_jobs=N_JOBS
    )

    # apply all needed masks by multiplying them to the image
    images_masked = images * body_masks if len(images) > 0 else np.array([])

    generate_vids_export_csv_one_minute(
        video_name_short, minute, 
        csv_writers, 
        images_masked, 
        generate_videos, 
        df_minute=df_minute_after
    )

    video_manager.clear_video_cache()



def process_video(
    video_filename: str, 
    df_video: pd.DataFrame, 
    generate_videos: bool
):
    """Process one original video. Exports derived feature to CSV-file

    Args:
        video_filename (str): Original video file (including path)
        df_video (pd.DataFrame): Dateframe for original video
        generate_videos (bool): Whether to generate video clips
    """

    video_name_short = (str(video_filename)).rsplit('/', 1)[1].split('.')[0]
    logger.info(f"Processing video {video_filename} ...")

    df_video = interpolate_missing_vals(df_video)
    
    df_video_grouped = df_video.groupby('bee_id', sort=False)
    sizes = df_video_grouped.size()
    # some video-datasets have a different number of rows for each bee, use last common frame-id
    if sizes.nunique() != 1:
        max_common_frame_index = df_video_grouped.frame_index.last().min()
        df_video = df_video[df_video.frame_index <= max_common_frame_index].copy()
        logger.warning(
            f"Different amounts of data for the bees detected! \
            Only extracting videos until frame index {max_common_frame_index} for each bee."
        )
        
    n_rows = len(df_video)
    logger.info(f"Total rows for {video_name_short}: {len(df_video)}")

    rows_per_bee = int(n_rows / 4)
    minutes = math.ceil(rows_per_bee / (60 * FRAME_RATE))
    logger.info(f"Minutes to process: {minutes}")
    logger.info(f"Rows per bee (total framecount in exported videos): {rows_per_bee}")
    remainder_last_minute = rows_per_bee % FRAMES_PER_MINUTE
    logger.debug(f"Remainder frames per bee in the last minute: {remainder_last_minute}")

    seconds_start = MINUTES_CUTOFF_START * 60
    seconds_end = math.ceil(seconds_start + (minutes - 1) * 60 + ((remainder_last_minute - 1) / FRAME_RATE))

    with open(MARKER_DATA_OUTPUT_PATH + video_name_short + "_bee0_" + (str(seconds_start) + "--" + str(seconds_end)) +
                ".csv", 'w', newline='') as csvfile_0, \
        open(MARKER_DATA_OUTPUT_PATH + video_name_short + "_bee1_" + (str(seconds_start) + "--" + str(seconds_end)) + 
                ".csv", 'w', newline='') as csvfile_1, \
        open(MARKER_DATA_OUTPUT_PATH + video_name_short + "_bee2_" + (str(seconds_start) + "--" + str(seconds_end)) + 
                ".csv", 'w', newline='') as csvfile_2, \
        open(MARKER_DATA_OUTPUT_PATH + video_name_short + "_bee3_" + (str(seconds_start) + "--" + str(seconds_end)) + 
                ".csv", 'w', newline='') as csvfile_3, \
        open(MARKER_DATA_OUTPUT_PATH + video_name_short + "_deriv_features_" + (str(seconds_start) + "--" + str(seconds_end)) + 
                ".csv", 'w', newline='') as csvfile_deriv:
        writer0 = csv.writer(csvfile_0, delimiter=',')
        writer1 = csv.writer(csvfile_1, delimiter=',')
        writer2 = csv.writer(csvfile_2, delimiter=',')
        writer3 = csv.writer(csvfile_3, delimiter=',')
        writer_deriv = csv.writer(csvfile_deriv, delimiter=',')
        csv_writers = [writer0, writer1, writer2, writer3]
        
        writer0.writerows([['scorer'] + [''] * 15, BODYPARTS_HEADER, COORDS_HEADER])
        writer1.writerows([['scorer'] + [''] * 15, BODYPARTS_HEADER, COORDS_HEADER])
        writer2.writerows([['scorer'] + [''] * 15, BODYPARTS_HEADER, COORDS_HEADER])
        writer3.writerows([['scorer'] + [''] * 15, BODYPARTS_HEADER, COORDS_HEADER])


        writer_deriv.writerow(['frame', 'bee_id', 'accel'])
        # is already sorted by bee and time
        for bee_id, df_bee_video in df_video.groupby('bee_id', sort=False):
            data = df_bee_video[['accel']].to_numpy()
            data_rows = [[i, bee_id, *row_arr] for i, row_arr in enumerate(data)]
            writer_deriv.writerows(data_rows)


        for minute in range(minutes):
            process_minute(
                video_filename, 
                video_name_short, 
                df_video, 
                rows_per_bee, 
                remainder_last_minute, 
                csv_writers, 
                minute, 
                generate_videos,
                total_minutes=minutes
            )
    
df_grouped = df.groupby('video_filename', sort=False)
for video_filename, df_video in df_grouped:
    process_video(video_filename, df_video, GENERATE_VIDEOS)

# Merge videos
Merge the minute-long videos of individuals into full videos

In [ ]:
if GENERATE_VIDEOS:
    logger.info("Merging videos ...")
    files = sorted(glob.glob(VIDEO_OUTPUT_PATH + 'hex_*.mp4'))

    video_part_map = {}
    command = "ffmpeg"
    path_to_command = "/usr/local/bin/"
    command = path_to_command + command
    input_videos_path = VIDEO_OUTPUT_PATH
    merge_videos_path = "/mnt/windows-ssd/trophallaxis_detection_code/data/"
    merge_txt_file = merge_videos_path + "individual_videos_merged_konstanz.txt"
    merge_videos_output_path = merge_videos_path + "individual_videos_merged_konstanz/"

    if not os.path.exists(merge_videos_output_path):
        os.makedirs(merge_videos_output_path)

    def create_merge_text_file(video_names):
        logger.info(video_names)
        with open(merge_txt_file, 'w') as f:
            for video_name in video_names:
                f.write("file " + video_name + os.linesep)

    for file in files:
        file_parts = file.rsplit('_', 1)
        start_frame = int((file_parts[1]).split('--')[0])
        unique_file_part = file_parts[0]
        video_part_map.setdefault(unique_file_part, SortedDict({})).update({start_frame: file})

    for k, v in video_part_map.items():
        create_merge_text_file(v.values())
        with open(merge_txt_file, 'r+') as f:
            lines = f.readlines()
            first_line = str(lines[0]).rstrip()
            last_line = str(lines[-1]).rstrip()
            output_path = merge_videos_output_path
            output_path += (first_line.split(input_videos_path)[1]).split("--")[0]
            output_path += "--" + (last_line.split("--")[1])
            if os.path.isfile(output_path):
                os.remove(output_path)
            call_args = ["-f", "concat", "-safe", "0", "-i", merge_txt_file,
                "-c", "copy", output_path]
            p = subprocess.Popen([command] + call_args, stderr=subprocess.PIPE)
            stdout, stderr = p.communicate()
            if p.returncode != 0:
                # an error happened!
                err_msg = "%s. Code: %s" % (stderr.strip(), p.returncode)
                raise Exception(err_msg)
            logger.info("Merging videos complete")

        